In [40]:
import poplib
from email.parser import Parser

import imaplib
import email
from email.header import decode_header

import os
from datetime import datetime, timedelta

In [44]:
# Configuración de la cuenta de Gmail
email_user = "alvarosaezsanchez@gmail.com"
email_pass = "eham tafy idvf veby"

# Carpeta local para descargar archivos adjuntos
download_folder = "./files"

# Conexión al servidor IMAP de Gmail
mail = imaplib.IMAP4_SSL("imap.gmail.com")
mail.login(email_user, email_pass)

# Selecciona el buzón de entrada (inbox)
mail.select("inbox")

# Construye la cadena de búsqueda con el remitente específico
remitente_a_buscar = "alvarosaezsanchez@gmail.com"
fecha_desde = (datetime.now() - timedelta(days=2)).strftime("%d-%b-%Y") #desde ayer
fecha_hoy = datetime.now().strftime("%d-%b-%Y")
asusto_a_buscar = "test gmail csv"
nombre_adjunto_a_buscar = "GEO_IP_1_comb (1).xlsx"
print(fecha_hoy)
criterio_busqueda = f'(UNSEEN FROM "{remitente_a_buscar}" SINCE {fecha_desde}) TEXT "{nombre_adjunto_a_buscar}"' #ON {fecha_a_buscar}' SUBJECT "{asunto_a_buscar}"

# Busca todos los correos no leídos
status, messages = mail.search(None,criterio_busqueda)

# Obtén los identificadores de los mensajes
mail_ids = messages[0].split()

for mail_id in mail_ids:
    # Obtiene el correo electrónico con el ID
    _, msg_data = mail.fetch(mail_id, "(RFC822)")
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            # Parsea el mensaje utilizando la librería email
            email_message = email.message_from_bytes(response_part[1])

            # Obtiene el asunto y el remitente
            subject, encoding = decode_header(email_message["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")
            from_, encoding = decode_header(email_message.get("From"))[0]
            if isinstance(from_, bytes):
                from_ = from_.decode(encoding if encoding else "utf-8")

            # Imprime el asunto y el remitente
            print("Subject:", subject)
            print("From:", from_)

            # Si el mensaje es multipart (contiene partes diferentes, como texto y adjuntos)
            if email_message.is_multipart():
                for part in email_message.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True)
                        print("Body:", body.decode("utf-8"))
                    else:
                        # Guarda el archivo CSV en la carpeta local
                        filename = part.get_filename()
                        if filename:
                            filepath = os.path.join(download_folder, filename)
                            with open(filepath, "wb") as f:
                                f.write(part.get_payload(decode=True))
                            print(f"Archivo guardado en: {filepath}")

# Cierra la conexión con el servidor IMAP
mail.logout()

01-Dec-2023
Subject: test gmail excel
From: Alv Sz <alvarosaezsanchez@gmail.com>
Body: test gmail excel

Archivo guardado en: ./files\GEO_IP_1_comb (1).xlsx


('BYE', [b'LOGOUT Requested'])